In [1]:
import os

In [2]:
%pwd

'c:\\ML_Projects\\MLflow_Implementation\\MLFlow-End-to-End-implementation\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\ML_Projects\\MLflow_Implementation\\MLFlow-End-to-End-implementation'

In [4]:
%cd MLFlow-End-to-End-implementation

[WinError 2] The system cannot find the file specified: 'MLFlow-End-to-End-implementation'
c:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\research


c:\ML_Projects\MLflow_Implementation\mlflow_implementation_project_env\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir: Path

In [6]:
from src.mlflowProject.constants import *
from src.mlflowProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.mlflowProject import logger
from src.mlflowProject.utils.common import get_size

In [9]:
class DataIngestion :
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with following info: \n{headers}")

        else:
            logger.info(f"File already exist of size: {get_size(self.config.local_data_file)}")

    
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file path
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion  = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e



[2024-10-28 22:51:27,975: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\config\config.yaml]
[2024-10-28 22:51:27,981: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\params.yaml]
[2024-10-28 22:51:27,984: INFO: common: json file loaded sucessfully from: C:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation\schema.yaml]
[2024-10-28 22:51:27,988: INFO: common: created directory at artifacts]
[2024-10-28 22:51:27,991: INFO: common: created directory at artifacts/data_ingestion]
[2024-10-28 22:51:29,454: INFO: 1433702343: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 199981
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6139955f7eee759c027db7c6f7c13fd28b2657fbc90d6ed28074cdd76dbb23f

In [11]:
import os
print(os.getcwd())


c:\ML_Projects\MLflow_Implementation\MLFlow-End-to-End-implementation


In [12]:
print(CONFIG_FILE_PATH.exists()) 


True
